In [44]:
import re
import time
import numpy as np
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import psycopg2
from sqlalchemy import create_engine

In [45]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz 

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz (15.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


In [46]:
# Загрузка модели для английского языка
nlp = spacy.load("en_core_sci_sm")

In [47]:
# Read the CSV files into dataframes
voc1_df = pd.read_csv('meddra_pt.csv')
voc2_df = pd.read_csv('SNOMED_OMOPext_CPT4_HCPCS.csv')
voc1_df=voc1_df[:10] # следующая 2000:4000
voc1_df = voc1_df.reset_index(drop=True)

In [48]:
voc1_df

,hierarchy,concept_id,concept_code,concept_name,domain_id,concept_class_id
0,"Neoplasms benign, malignant and unspecified (i...",36617673,10047746,Vulval cancer stage I,Condition,PT
1,"Neoplasms benign, malignant and unspecified (i...",36617674,10047747,Vulval cancer stage II,Condition,PT
2,"Neoplasms benign, malignant and unspecified (i...",36617675,10047748,Vulval cancer stage III,Condition,PT
3,"Neoplasms benign, malignant and unspecified (i...",36617676,10047749,Vulval cancer stage IV,Condition,PT
4,Surgical and medical procedures(SOC)-Obstetric...,37521584,10047772,Vulval warts removal,Procedure,PT
5,"Neoplasms benign, malignant and unspecified (i...",36150289,10047793,Vulvovaginal warts,Condition,PT
6,Blood and lymphatic system disorders(SOC)-Neop...,35104568,10047804,Waldenstrom's macroglobulinaemia recurrent,Condition,PT
7,Blood and lymphatic system disorders(SOC)-Neop...,35104569,10047805,Waldenstrom's macroglobulinaemia refractory,Condition,PT
8,Blood and lymphatic system disorders(SOC)-Neop...,35104570,10047806,Waldenstrom's macroglobulinaemia stage I,Condition,PT
9,Blood and lymphatic system disorders(SOC)-Neop...,35104571,10047807,Waldenstrom's macroglobulinaemia stage II,Condition,PT


In [49]:
voc2_df

,concept_id,concept_name,concept_name_processed
0,36684768,Drusen of bilateral optic discs,"['drusen', 'of', 'bilateral', 'optic', 'discs']"
1,37110249,Microvascular embolism of arteriole (disorder),"['microvascular', 'embolism', 'of', 'arteriole']"
2,4220821,Bronzed diabetes,"['bronze', 'diabete']"
3,4120412,Level of psychoticism,"['level', 'of', 'psychoticism']"
4,4002835,Bruising of oropharynx,"['bruising', 'of', 'oropharynx']"
...,...,...,...
564483,4036092,ROM - Range of motion activity,"['rom', 'range', 'of', 'motion', 'activity']"
564484,37309624,Low back pain co-occurrent with left side scia...,"['low', 'back', 'pain', 'co', 'occurrent', 'wi..."
564485,4006806,Miscarriage with uraemia,"['miscarriage', 'with', 'uraemia']"
564486,4238036,Urological fistula,"['urological', 'fistula']"


In [50]:
# # words to remove from strings
# stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd",
#              'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
#              'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what',
#              'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were',
#              'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
#              'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about',
#              'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from',
#              'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
#              'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
#              'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
#              'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain',
#              'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
#              "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn',
#              "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't",
#              'wouldn', "wouldn't",
#               # don't care  about right and left most of the time
#              #'right','left']

In [75]:
# Stop_words that were received from SciSpacy model (UMLS based)
stop_words = ["'d", "'ll", "'m", "'re", "'s", "'ve", 'a', 'about', 'above', 'across', 'after', 'afterwards', 'again',
              'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 
              'amongst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 
              'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 
              'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 
              'between', 'beyond', 'both', 'bottom', 'but', 'by', 'ca', 'call', 'can', 'cannot', 'could', 'did', 
              'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 
              'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 
              'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 
              'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 
              'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 
              'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 
              'last', 'latter', 'latterly', 'least', 'less', 'made', 'make', 'many', 'may', 'me', 'meanwhile', 
              'might', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', "n't", 
              'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'none', 'noone',
              'nor', 'not', 'nothing', 'now', 'nowhere', 'n‘t', 'n’t', 'of', 'off', 'often', 'on', 'once', 'one', 
              'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own',
              'part', 'per', 'perhaps', 'please', 'put', 'quite', 'rather', 're', 'really', 'regarding', 'same',
              'say', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show',
              'side', 'since', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 'sometime',
              'sometimes', 'somewhere', 'still', 'such', 'take', 'ten', 'than', 'that', 'the', 'their', 'them', 
              'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon',
              'these', 'they', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 
              'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'under', 'unless', 
              'until', 'up', 'upon', 'us', 'used', 'using', 'various', 'very', 'via', 'was', 'we', 'well', 'were', 
              'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby',
              'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole',
              'whom', 'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours',
              'yourself', 'yourselves', '‘d', '‘ll', '‘m', '‘re', '‘s', '‘ve', '’d', '’ll', '’m', '’re', '’s', '’ve']

# aditional task-specific stop_words

stop_words = stop_words + ['noc', 'nos', '[d]', 'unknown_unit', '|', 'see comment', 'due', 'nec', 'unspecified', '[v]', '(see comments)',
              '(disorder)', '(procedure)', '(finding)', '(observable entity)']

In [76]:
# Convert the titles to lowercase
voc1_df['concept_name_processed'] = voc1_df['concept_name'].map(lambda x: str(x).lower())
voc2_df['concept_name_processed'] = voc2_df['concept_name'].map(lambda x: str(x).lower())


# Remove stopwords
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


# ignores weird symbols
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: re.sub(r"\s+", " ", str(x).encode('ascii', 'ignore').decode()) if isinstance(x, str) else str(x))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: re.sub(r"\s+", " ", str(x).encode('ascii', 'ignore').decode()) if isinstance(x, str) else str(x))

# заменить '-' на ' '
#voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].map(lambda x: re.sub('-', ' ', x))
#voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].map(lambda x: re.sub('-', ' ', x))

# заменить '/' на ' '
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].map(lambda x: re.sub('/', ' ', x))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].map(lambda x: re.sub('/', ' ', x))


# Remove punctuation, digits etc
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].map(lambda x: re.sub('[,\.!;?)%(\'\":]', '', x))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].map(lambda x: re.sub('[,\.!;?)%(\'\":]', '', x))


In [52]:
voc1_df.head(5)

,hierarchy,concept_id,concept_code,concept_name,domain_id,concept_class_id,concept_name_processed
0,"Neoplasms benign, malignant and unspecified (i...",36617673,10047746,Vulval cancer stage I,Condition,PT,vulval cancer stage i
1,"Neoplasms benign, malignant and unspecified (i...",36617674,10047747,Vulval cancer stage II,Condition,PT,vulval cancer stage ii
2,"Neoplasms benign, malignant and unspecified (i...",36617675,10047748,Vulval cancer stage III,Condition,PT,vulval cancer stage iii
3,"Neoplasms benign, malignant and unspecified (i...",36617676,10047749,Vulval cancer stage IV,Condition,PT,vulval cancer stage iv
4,Surgical and medical procedures(SOC)-Obstetric...,37521584,10047772,Vulval warts removal,Procedure,PT,vulval warts removal


In [53]:
voc2_df.head(5)

,concept_id,concept_name,concept_name_processed
0,36684768,Drusen of bilateral optic discs,"['drusen', 'of', 'bilateral', 'optic', 'discs']"
1,37110249,Microvascular embolism of arteriole (disorder),"['microvascular', 'embolism', 'of', 'arteriole']"
2,4220821,Bronzed diabetes,"['bronze', 'diabete']"
3,4120412,Level of psychoticism,"['level', 'of', 'psychoticism']"
4,4002835,Bruising of oropharynx,"['bruising', 'of', 'oropharynx']"


In [54]:
def tokenize_with_spacy(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
    return tokens

start_time = time.time()

voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(tokenize_with_spacy)
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(tokenize_with_spacy)

elapsed_time = (time.time() - start_time)/60
print(f"Total elapsed time for tokenization: {elapsed_time:.1f} minutes")


def lemmatize_with_spacy(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return lemmatized_tokens

start_time = time.time()

# Применение функции к вашему столбцу
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: lemmatize_with_spacy(' '.join(x)))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: lemmatize_with_spacy(' '.join(x)))

elapsed_time = (time.time() - start_time)/60
print(f"Total elapsed time for lemmatization: {elapsed_time:.1f} minutes")




Total elapsed time for tokenization: 0.0 minutes
Total elapsed time for lemmatization: 0.0 minutes


In [55]:
voc1_df.head()

,hierarchy,concept_id,concept_code,concept_name,domain_id,concept_class_id,concept_name_processed
0,"Neoplasms benign, malignant and unspecified (i...",36617673,10047746,Vulval cancer stage I,Condition,PT,"[vulval, cancer, stage, I]"
1,"Neoplasms benign, malignant and unspecified (i...",36617674,10047747,Vulval cancer stage II,Condition,PT,"[vulval, cancer, stage, ii]"
2,"Neoplasms benign, malignant and unspecified (i...",36617675,10047748,Vulval cancer stage III,Condition,PT,"[vulval, cancer, stage, iii]"
3,"Neoplasms benign, malignant and unspecified (i...",36617676,10047749,Vulval cancer stage IV,Condition,PT,"[vulval, cancer, stage, iv]"
4,Surgical and medical procedures(SOC)-Obstetric...,37521584,10047772,Vulval warts removal,Procedure,PT,"[vulval, wart, removal]"


In [56]:
voc2_df.head()

,concept_id,concept_name,concept_name_processed
0,36684768,Drusen of bilateral optic discs,"['drusen', 'of', 'bilateral', 'optic', 'discs']"
1,37110249,Microvascular embolism of arteriole (disorder),"['microvascular', 'embolism', 'of', 'arteriole']"
2,4220821,Bronzed diabetes,"['bronze', 'diabete']"
3,4120412,Level of psychoticism,"['level', 'of', 'psychoticism']"
4,4002835,Bruising of oropharynx,"['bruising', 'of', 'oropharynx']"


In [14]:
# Save previously prepared vocabulary for future use
voc2_df.to_csv('SNOMED_OMOPext_CPT4_HCPCS_preprocessed.csv', index=False)

# TfIDF

In [15]:
# Preprocess and compute the TF-IDF document-term matrix for the snomed concepts
voc2_docs = voc2_df['concept_name_processed'].apply(lambda x: ''.join(x)).tolist()
tfidf_vectorizer = TfidfVectorizer()
voc2_dtm = tfidf_vectorizer.fit_transform(voc2_docs)

# Calculate cosine similarities and find 5 most similar snomed concepts for each meddra concept
results = []

batch_size = 10
num_top_matches = 10

start_time = time.time()
processed_count = 0

for voc1_index, voc1_row in voc1_df.iterrows():
    voc1_tokens = voc1_row['concept_name_processed']
    voc1_name = voc1_row['concept_name']
    voc1_concept_id = voc1_row['concept_id']

    voc1_text = ' '.join(voc1_tokens)
    voc1_dtm = tfidf_vectorizer.transform([voc1_text])

    similarities = cosine_similarity(voc1_dtm, voc2_dtm)[0]

    # Find indices of 5 most similar voc2 (SNOMED and etc.) concepts
    top_indices = np.argsort(similarities)[-num_top_matches:][::-1]

    for max_index in top_indices:
        voc2_concept_id = voc2_df.loc[max_index, 'concept_id']
        voc2_name = voc2_df.loc[max_index, 'concept_name']
        similarity = similarities[max_index]

        # Add a separate line of information to the results
        results.append((voc1_name, voc1_concept_id, voc2_concept_id, voc2_name, similarity))

    processed_count += 1
    if processed_count % batch_size == 0:
        elapsed_time = (time.time() - start_time) / 60
        print(f"Processed {processed_count} concepts in {elapsed_time:.1f} minutes")

# Sort the results by similarity in descending order
results.sort(key=lambda x: x[4], reverse=True)

# Convert results to DataFrame
columns = ['voc1_name', 'voc1_concept_id', 'voc2_concept_id', 'voc2_name', 'similarity']
results_df = pd.DataFrame(results, columns=columns)

elapsed_time = (time.time() - start_time) / 60
print(f"Total elapsed time: {elapsed_time:.1f} minutes")

Processed 10 concepts in 0.0 minutes
Processed 20 concepts in 0.0 minutes
Processed 30 concepts in 0.0 minutes
Processed 40 concepts in 0.0 minutes
Processed 50 concepts in 0.1 minutes
Processed 60 concepts in 0.1 minutes
Processed 70 concepts in 0.1 minutes
Processed 80 concepts in 0.1 minutes
Processed 90 concepts in 0.1 minutes
Processed 100 concepts in 0.1 minutes
Processed 110 concepts in 0.1 minutes
Processed 120 concepts in 0.1 minutes
Processed 130 concepts in 0.1 minutes
Processed 140 concepts in 0.2 minutes
Processed 150 concepts in 0.2 minutes
Processed 160 concepts in 0.2 minutes
Processed 170 concepts in 0.2 minutes
Processed 180 concepts in 0.2 minutes
Processed 190 concepts in 0.2 minutes
Processed 200 concepts in 0.2 minutes
Processed 210 concepts in 0.2 minutes
Processed 220 concepts in 0.2 minutes
Processed 230 concepts in 0.3 minutes
Processed 240 concepts in 0.3 minutes
Processed 250 concepts in 0.3 minutes
Processed 260 concepts in 0.3 minutes
Processed 270 concept

In [16]:
# Output the results as a Pandas dataframe
df_sorted = results_df.sort_values(by=['voc1_name', 'similarity'], ascending=False)
df_sorted['origin']='TfIDF'
df_sorted.to_csv('final_table'+'_tfidf.csv', index=False)
df_sorted.head(50)

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,similarity,origin
3289,Vulval cancer recurrent,36617671,195197,Primary vulval cancer,0.777719,TfIDF
4769,Vulval cancer recurrent,36617671,764225,Recurrent cancer,0.740397,TfIDF
14219,Vulval cancer recurrent,36617671,442259,Vulval disorder,0.583841,TfIDF
14502,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer (disorder),0.579575,TfIDF
14503,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer,0.579575,TfIDF
15389,Vulval cancer recurrent,36617671,4172084,Vulval biopsy,0.564193,TfIDF
16114,Vulval cancer recurrent,36617671,4012199,Vulval pain (finding),0.551910,TfIDF
16115,Vulval cancer recurrent,36617671,4012199,Vulval pain,0.551910,TfIDF
16323,Vulval cancer recurrent,36617671,606881,At risk of recurrent cancer,0.547587,TfIDF
16496,Vulval cancer recurrent,36617671,434084,Vulval abscess,0.544525,TfIDF


In [17]:
df_tfidf=df_sorted

# Fuzz_ratio

**Rapidfuzz.fuzz.ratio** is a function from the rapidfuzz library in Python, which is used for string matching and similarity comparisons. The rapidfuzz library is known for its efficient and fast computation of string similarities, often utilized in scenarios where large datasets require rapid processing of string comparison tasks.

The fuzz.ratio function computes the similarity between two strings, returning a score that represents how similar the two strings are. The score is an integer ranging from 0 to 100, where 100 indicates an exact match (the strings are identical), and lower scores indicate less similarity.

This function implements an algorithm similar to the Levenshtein Distance, which calculates the number of single-character edits (insertions, deletions, or substitutions) required to change one word into the other. However, rapidfuzz has optimized the computation for performance, making it faster than many other implementations of similar algorithms.

In summary, rapidfuzz.fuzz.ratio is a fast and efficient way to compare two strings and quantify their similarity, widely used in tasks like data cleaning, deduplication, and matching in various applications.

In [57]:
#!pip install python-Levenshtein

In [58]:
#import Levenshtein as lev

In [59]:
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: sorted(x))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: x.strip("[]").replace("'", "").split(', '))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: sorted(x))

In [60]:
voc2_df.head(10)

,concept_id,concept_name,concept_name_processed
0,36684768,Drusen of bilateral optic discs,"[bilateral, discs, drusen, of, optic]"
1,37110249,Microvascular embolism of arteriole (disorder),"[arteriole, embolism, microvascular, of]"
2,4220821,Bronzed diabetes,"[bronze, diabete]"
3,4120412,Level of psychoticism,"[level, of, psychoticism]"
4,4002835,Bruising of oropharynx,"[bruising, of, oropharynx]"
5,4336536,Dilation of nasopharynx (procedure),"[dilation, nasopharynx, of]"
6,4232889,Discharge readiness: supported living,"[discharge, live, readiness, support]"
7,4013513,Cardiovascular measurement,"[cardiovascular, measurement]"
8,36715972,Human herpesvirus 4 meningitis,"[4, herpesvirus, human, meningitis]"
9,4018600,"Complement C1 esterase inhibitor, total measur...","[c1, complement, esterase, inhibitor, measurem..."


In [61]:
# # Old variant of Levenshtein
# start_time = time.time()
# results = []

# count = 0
# num_top_matches = 10  # Указываем количество ближайших совпадений, которые мы хотим сохранить

# for voc1_index, voc1_row in voc1_df.iterrows():
#     voc1_name = voc1_row['concept_name']
#     voc1_concept_id=voc1_row['concept_id']
#     voc1_tokens = voc1_row['concept_name_processed']

#     min_distances = [float('inf')] * num_top_matches  # Список для хранения наименьших расстояний
#     top_matches = [[] for _ in range(num_top_matches)]  # Список для хранения ближайших сочетаний

#     for voc2_index, voc2_row in voc2_df.iterrows():
#         voc2_concept_id = voc2_row['concept_id']
#         voc2_name = voc2_row['concept_name_processed']

#         # Calculate Levenshtein distance between voc1 and voc2 concept names
#         distance = lev.distance(''.join(voc1_tokens), ''.join(voc2_name))

#         # Обновляем ближайшие совпадения, если найдено более близкое совпадение
#         for i in range(num_top_matches):
#             if distance < min_distances[i]:
#                 min_distances.insert(i, distance)
#                 min_distances.pop()
#                 top_matches.insert(i, (voc2_concept_id, voc2_row['concept_name'], distance))
#                 top_matches.pop()
#                 break

#     # Сортируем top_matches по возрастанию min_distance перед добавлением в results
#     top_matches.sort(key=lambda x: x[2])

#     # Добавляем ближайшие совпадения в результаты
#     for matches in top_matches:
#         if matches:  # Проверяем, что список не пустой
#             voc2_concept_id, voc2_name, min_distance = matches
#             results.append((voc1_name, voc1_concept_id, voc2_concept_id, voc2_name, min_distance))

#     count += 1
#     if count % 10 == 0:
#         elapsed_time = (time.time() - start_time) / 60
#         print(f"Elapsed time: {elapsed_time:.1f} minutes for {count} концептов")

# # Convert results to DataFrame
# columns = ['voc1_name', 'voc1_concept_id', 'voc2_concept_id', 'voc2_name', 'min_distance']
# results_df = pd.DataFrame(results, columns=columns)

# elapsed_time = (time.time() - start_time) / 60
# print(f"Total elapsed time: {elapsed_time:.1f} minutes")

Elapsed time: 10.0 minutes for 10 концептов
Elapsed time: 20.4 minutes for 20 концептов
Elapsed time: 30.8 minutes for 30 концептов
Total elapsed time: 30.8 minutes


In [62]:
# # Output the results as a Pandas dataframe
# df_sorted = results_df.sort_values(by=['voc1_name', 'min_distance'], ascending=True)
# df_sorted['origin']='Levenshtein_distance'
# df_sorted.to_csv('final_table'+'_levenshtein_old.csv', index=False)
# df_sorted.head(50)

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,min_distance,origin
0,Vulval cancer stage I,36617673,4178959,Cancer of vulva,7,Levenshtein_distance
1,Vulval cancer stage I,36617673,195197,Primary vulval cancer,7,Levenshtein_distance
2,Vulval cancer stage I,36617673,4195031,Taking vulval swab (procedure),8,Levenshtein_distance
3,Vulval cancer stage I,36617673,40480197,Interval cancer,8,Levenshtein_distance
4,Vulval cancer stage I,36617673,4294557,Vulval chancroid (disorder),8,Levenshtein_distance
5,Vulval cancer stage I,36617673,4195031,Taking vulval swab,8,Levenshtein_distance
6,Vulval cancer stage I,36617673,4294557,Vulval chancroid,8,Levenshtein_distance
7,Vulval cancer stage I,36617673,198984,Cervical cancer,9,Levenshtein_distance
8,Vulval cancer stage I,36617673,4221654,Vulval eczema (disorder),9,Levenshtein_distance
9,Vulval cancer stage I,36617673,443398,Intestinal cancer,9,Levenshtein_distance


In [63]:
!pip install rapidfuzz
import rapidfuzz

In [67]:
# new version from ChatGPT
import time
import pandas as pd
import numpy as np
from rapidfuzz import process
import concurrent.futures

start_time = time.time()

num_top_matches = 10

# Preprocessing
voc1_df['voc1_tokens_str'] = voc1_df['concept_name_processed'].apply(''.join)
voc2_names = voc2_df['concept_name_processed'].apply(''.join).tolist()

# Define a function to process a chunk of rows
def process_chunk(chunk, voc2_names):
    results = []
    for _, voc1_row in chunk.iterrows():
        voc1_name = voc1_row['concept_name']
        voc1_concept_id = voc1_row['concept_id']
        voc1_tokens_str = voc1_row['voc1_tokens_str']

        # Get top matches using rapidfuzz
        top_matches = process.extract(voc1_tokens_str, voc2_names, limit=num_top_matches, scorer=rapidfuzz.fuzz.ratio)
        
        # Unpack the results correctly
        for match in top_matches:
            matched_string, score, index = match
            voc2_concept_id = voc2_df.iloc[index]['concept_id']
            voc2_name = voc2_df.iloc[index]['concept_name']
            results.append((voc1_name, voc1_concept_id, voc2_concept_id, voc2_name, score))
    
    return results

# Split DataFrame into chunks for parallel processing
chunk_size = 100  # Adjust this based on your DataFrame's size and available memory
chunks = [voc1_df.iloc[i:i + chunk_size] for i in range(0, voc1_df.shape[0], chunk_size)]

# Parallel processing of chunks
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_chunk, chunks, [voc2_names] * len(chunks)))

# Flatten the list of results
flattened_results = [item for sublist in results for item in sublist]

# Convert results to DataFrame
columns = ['voc1_name', 'voc1_concept_id', 'voc2_concept_id', 'voc2_name', 'fuzz_ration']
results_df = pd.DataFrame(flattened_results, columns=columns)

elapsed_time = (time.time() - start_time) / 60
print(f"Total elapsed time: {elapsed_time:.1f} minutes")



Total elapsed time: 0.1 minutes


In [68]:
results_df

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,fuzz_ration
0,Vulval cancer stage I,36617673,4116086,Paget disease vulvar cancer,71.428571
1,Vulval cancer stage I,36617673,4178959,Cancer of vulva,70.967742
2,Vulval cancer stage I,36617673,4181343,Esophageal cancer,70.588235
3,Vulval cancer stage I,36617673,195197,Primary vulval cancer,70.270270
4,Vulval cancer stage I,36617673,438386,Cancer in situ of vulva,70.270270
...,...,...,...,...,...
295,X-ray of pelvis and hip abnormal,36314881,4056699,Plain X-ray hand abnormal,70.833333
296,X-ray of pelvis and hip abnormal,36314881,44783486,"MRI of thorax, abdomen and pelvis",70.370370
297,X-ray of pelvis and hip abnormal,36314881,4167549,"CT of chest, abdomen and pelvis",69.387755
298,X-ray of pelvis and hip abnormal,36314881,4056705,Plain X-ray hip joint abnormal,69.230769


In [70]:
# Output the results as a Pandas dataframe
df_sorted = results_df.sort_values(by=['voc1_name', 'fuzz_ratio'], ascending=False)
df_sorted['origin']='fuzz_ratio'
df_sorted.to_csv('final_table'+'_fuzz_ratio.csv', index=False)
df_sorted.head(50)

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,fuzz_ration,origin
290,X-ray of pelvis and hip abnormal,36314881,4222906,X-ray of pelvis and hip,75.555556,Levenshtein_distance
291,X-ray of pelvis and hip abnormal,36314881,37016203,MRI of pelvis abnormal,73.913043,Levenshtein_distance
292,X-ray of pelvis and hip abnormal,36314881,37018682,CT of pelvis abnormal,71.111111,Levenshtein_distance
293,X-ray of pelvis and hip abnormal,36314881,4170261,MRI of abdomen and pelvis,70.833333,Levenshtein_distance
294,X-ray of pelvis and hip abnormal,36314881,4056699,Plain X-ray hand abnormal (finding),70.833333,Levenshtein_distance
295,X-ray of pelvis and hip abnormal,36314881,4056699,Plain X-ray hand abnormal,70.833333,Levenshtein_distance
296,X-ray of pelvis and hip abnormal,36314881,44783486,"MRI of thorax, abdomen and pelvis",70.370370,Levenshtein_distance
297,X-ray of pelvis and hip abnormal,36314881,4167549,"CT of chest, abdomen and pelvis",69.387755,Levenshtein_distance
298,X-ray of pelvis and hip abnormal,36314881,4056705,Plain X-ray hip joint abnormal,69.230769,Levenshtein_distance
299,X-ray of pelvis and hip abnormal,36314881,4056705,Plain X-ray hip joint abnormal (finding),69.230769,Levenshtein_distance


In [24]:
df_fuzz_ratio=df_sorted

# BioWordVec

In [15]:
# Запускать, если не запускается перед этим Levenshtein:
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: sorted(x))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: x.strip("[]").replace("'", "").split(', '))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: sorted(x))

In [25]:
# Запускать, если перед этим уже был запущен Levenshtein

voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: sorted(x))
#voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: x.strip("[]").replace("'", "").split(', '))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: sorted(x))
#voc1_df['target_concept_id']=0

In [16]:
import time
import numpy as np
import gensim
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained word embeddings model
model_path = 'H:/BioWordVec_PubMed_MIMICIII_d200.vec.bin'
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)
start_time = time.time()

# Функция для векторизации текста с использованием BioWordVec
def vectorize_text(text, model):
    tokens = text.split()
    vectors = [model[token] for token in tokens if token in model]
    if vectors:
        return np.mean(vectors, axis=0)  # Усреднение векторов
    else:
        return np.zeros(model.vector_size)  # Возвращает нулевой вектор, если нет известных слов

# Применение функции к медицинским терминам в обоих наборах данных
voc1_df['voc1_vector'] = voc1_df['concept_name_processed'].apply(lambda x: vectorize_text(' '.join(x), word2vec_model))
voc2_df['voc2_vector'] = voc2_df['concept_name_processed'].apply(lambda x: vectorize_text(' '.join(x), word2vec_model))

# Удаление строк с отсутствующими векторами
voc1_df = voc1_df.dropna(subset=['voc1_vector'])
voc2_df = voc2_df.dropna(subset=['voc2_vector'])


In [17]:
start_time = time.time()

num_top_matches = 10

v1 = np.vstack(voc1_df['voc1_vector'])
v2 = np.vstack(voc2_df['voc2_vector'])
similarity = cosine_similarity(v1, v2)
tops = (-similarity).argsort()[:, :num_top_matches]

results_df = pd.concat(     # concat (axis = 0) the list of data frames
    [pd.concat( # generate a list of data frames by concatenating (axis = 1) selected rows and cols from voc1 and voc2
        [voc1_df.loc[[i], ['concept_name', 'concept_id']].reset_index(drop=True),  # for each target concept from voc_1
         voc2_df.loc[[t], ['concept_id', 'concept_name']].reset_index(drop=True),        # add potential mapping match from voc_2
         pd.Series(similarity[i][t], name='Similarity')], axis= 1)                       # and similarity value
     for i in range(0, len(voc1_df)) for t in tops[i]]                                   # `i` - target vocabulary index, `t` - index in top similarity array
).reset_index(drop=True)

elapsed_time = (time.time() - start_time) / 60
print(f"Total elapsed time: {elapsed_time:.1f} minutes")

Total elapsed time: 7.7 minutes


In [ ]:
# Old working but very slow code

# results = []
# num_top_matches = 10

# for voc1_index, voc1_row in voc1_df.iterrows():
#     voc1_name = voc1_row['concept_name']
#     voc1_vector = voc1_row['voc1_vector']
#     voc1_concept_id = voc1_row['concept_id']

#     similarities = []
#     for voc2_index, voc2_row in voc2_df.iterrows():
#         voc2_name = voc2_row['concept_name']  # Сохраняем concept_id
#         voc2_concept_id = voc2_row['concept_id']
#         voc2_vector = voc2_row['voc2_vector']

#         if voc1_vector is not None and voc2_vector is not None:
#             similarity = cosine_similarity([voc1_vector], [voc2_vector])[0][0]
#             similarities.append((voc2_name, voc2_concept_id, similarity))

#     similarities.sort(key=lambda x: x[2], reverse=True)
#     top_matches = similarities[:num_top_matches]

#     for voc2_name, voc2_concept_id, similarity in top_matches:
#         results.append((voc1_name, voc1_concept_id, voc2_concept_id, voc2_name, similarity))  # Добавляем voc1_index и voc2_name

#     # Счетчик каждых 10 обработанных строк
#     if (voc1_index + 1) % 10 == 0:
#         elapsed_time = (time.time() - start_time) / 60
#         print(f"Elapsed time: {elapsed_time:.1f} minutes for processed {voc1_index + 1} rows")

        
# # Создать DataFrame с результатами
# columns = ['voc1_name', 'voc1_concept_id', 'voc2_concept_id', 'voc2_name', 'similarity']  # Добавляем 'voc1_index'
# results_df = pd.DataFrame(results, columns=columns)

# elapsed_time = (time.time() - start_time) / 60
# print(f"Total elapsed time: {elapsed_time:.1f} minutes")

In [18]:
results_df.head(50)

,concept_name,concept_id,concept_id,concept_name,Similarity
0,17 ketosteroids urine,36312355,4324980,"17-Ketosteroids measurement, fractionation, ur...",0.913912
1,17 ketosteroids urine,36312355,4324980,"17-Ketosteroids measurement, fractionation, urine",0.913912
2,17 ketosteroids urine,36312355,4042261,Urine 17 ketogenic steroid measurement (proced...,0.902799
3,17 ketosteroids urine,36312355,4042261,Urine 17 ketogenic steroid measurement,0.902799
4,17 ketosteroids urine,36312355,40480493,High 17-ketosteroid level in urine,0.892626
5,17 ketosteroids urine,36312355,4042261,Urine 17 ketogenic steroid level,0.891024
6,17 ketosteroids urine,36312355,37393636,Urine 17 ketogenic steroid level,0.891024
7,17 ketosteroids urine,36312355,4246849,Urine 18-hydroxycortisol level (procedure),0.884305
8,17 ketosteroids urine,36312355,37393169,Urine 18-hydroxycortisol level,0.884305
9,17 ketosteroids urine,36312355,4246849,Urine 18-hydroxycortisol level,0.884305


In [19]:
# Modification to new variant
new_columns = list(results_df.columns)
new_columns[0] = 'voc1_name'
new_columns[1]='voc1_concept_id'
new_columns[2]='voc2_concept_id'
new_columns[3]='voc2_name'
new_columns[4]='similarity'
results_df.columns = new_columns


# Присвойте новый список названий столбцов атрибуту 'columns' DataFrame
#results_df['voc1_concept_id']=0
results_df

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,similarity
0,17 ketosteroids urine,36312355,4324980,"17-Ketosteroids measurement, fractionation, ur...",0.913912
1,17 ketosteroids urine,36312355,4324980,"17-Ketosteroids measurement, fractionation, urine",0.913912
2,17 ketosteroids urine,36312355,4042261,Urine 17 ketogenic steroid measurement (proced...,0.902799
3,17 ketosteroids urine,36312355,4042261,Urine 17 ketogenic steroid measurement,0.902799
4,17 ketosteroids urine,36312355,40480493,High 17-ketosteroid level in urine,0.892626
...,...,...,...,...,...
19995,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer (disorder),0.881319
19996,Vulval cancer recurrent,36617671,4312395,"Cancer of vulva, disseminated",0.860730
19997,Vulval cancer recurrent,36617671,45769825,Recurrent primary malignant neoplasm of vulva ...,0.859373
19998,Vulval cancer recurrent,36617671,45769825,Recurrent primary malignant neoplasm of vulva,0.859373


In [20]:
# Output the results as a Pandas dataframe
df_sorted = results_df.sort_values(by=['voc1_name', 'similarity'], ascending=False)
df_sorted['origin']='BioWordVec'
df_sorted.to_csv('final_table'+'_biowordvec.csv', index=False)
df_sorted.head(50)

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,similarity,origin
19990,Vulval cancer recurrent,36617671,195197,Primary vulval cancer,0.918315,BioWordVec
19991,Vulval cancer recurrent,36617671,4178959,Cancer of vulva,0.901247,BioWordVec
19992,Vulval cancer recurrent,36617671,764225,Recurrent cancer,0.899003,BioWordVec
19993,Vulval cancer recurrent,36617671,4313226,Cancer metastatic to vulva,0.885942,BioWordVec
19994,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer,0.881319,BioWordVec
19995,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer (disorder),0.881319,BioWordVec
19996,Vulval cancer recurrent,36617671,4312395,"Cancer of vulva, disseminated",0.860730,BioWordVec
19997,Vulval cancer recurrent,36617671,45769825,Recurrent primary malignant neoplasm of vulva ...,0.859373,BioWordVec
19998,Vulval cancer recurrent,36617671,45769825,Recurrent primary malignant neoplasm of vulva,0.859373,BioWordVec
19999,Vulval cancer recurrent,36617671,40487046,Carcinoma of vulva,0.855377,BioWordVec


In [23]:
df_biowordvec=df_sorted

# Final merge tables and remove duplicates

In [25]:
import datetime
current_datetime = datetime.datetime.now()
current_date = current_datetime.date()

In [26]:
df_tfidf = pd.read_csv('final_table_tfidf.csv')
df_tfidf.drop_duplicates(subset=['voc1_name', 'voc2_concept_id'], inplace=True)
df_tfidf

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,similarity,origin
0,Vulval cancer recurrent,36617671,195197,Primary vulval cancer,0.777719,TfIDF
1,Vulval cancer recurrent,36617671,764225,Recurrent cancer,0.740397,TfIDF
2,Vulval cancer recurrent,36617671,442259,Vulval disorder,0.583841,TfIDF
3,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer (disorder),0.579575,TfIDF
5,Vulval cancer recurrent,36617671,4172084,Vulval biopsy,0.564193,TfIDF
...,...,...,...,...,...,...
19990,17 ketosteroids urine,36312355,4018340,17-Ketosteroids measurement (procedure),0.878312,TfIDF
19992,17 ketosteroids urine,36312355,4035824,Disorder of 17-ketosteroids,0.858936,TfIDF
19993,17 ketosteroids urine,36312355,40480493,High 17-ketosteroid level in urine,0.850166,TfIDF
19996,17 ketosteroids urine,36312355,4017199,"17-Ketosteroids measurement, total (procedure)",0.809079,TfIDF


In [27]:
df_fuzz_ratio = pd.read_csv('final_table_fuzz_ratio.csv')
df_fuzz_ratio.drop_duplicates(subset=['voc1_name', 'voc2_concept_id'], inplace=True)
df_fuzz_ratio

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,min_distance,origin
0,17 ketosteroids urine,36312355,4055428,Urine steroid profile (procedure),7,Levenshtein_distance
1,17 ketosteroids urine,36312355,37393311,Urine steroid profile,7,Levenshtein_distance
3,17 ketosteroids urine,36312355,4150637,Urine ketone test,8,Levenshtein_distance
4,17 ketosteroids urine,36312355,37393634,Urine ketone test,8,Levenshtein_distance
5,17 ketosteroids urine,36312355,4055829,Urine ketone test = +,8,Levenshtein_distance
...,...,...,...,...,...,...
19995,Vulval cancer recurrent,36617671,443390,Rectal cancer,9,Levenshtein_distance
19996,Vulval cancer recurrent,36617671,198984,Cervical cancer,10,Levenshtein_distance
19997,Vulval cancer recurrent,36617671,440637,Recurrent fever,10,Levenshtein_distance
19998,Vulval cancer recurrent,36617671,4178959,Cancer of vulva,10,Levenshtein_distance


In [28]:
df_biowordvec = pd.read_csv('final_table_biowordvec.csv')
df_biowordvec.drop_duplicates(subset=['voc1_name', 'voc2_concept_id'], inplace=True)
df_biowordvec

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,similarity,origin
0,Vulval cancer recurrent,36617671,195197,Primary vulval cancer,0.918315,BioWordVec
1,Vulval cancer recurrent,36617671,4178959,Cancer of vulva,0.901247,BioWordVec
2,Vulval cancer recurrent,36617671,764225,Recurrent cancer,0.899003,BioWordVec
3,Vulval cancer recurrent,36617671,4313226,Cancer metastatic to vulva,0.885942,BioWordVec
4,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer,0.881319,BioWordVec
...,...,...,...,...,...,...
19992,17 ketosteroids urine,36312355,4042261,Urine 17 ketogenic steroid measurement (proced...,0.902799,BioWordVec
19994,17 ketosteroids urine,36312355,40480493,High 17-ketosteroid level in urine,0.892626,BioWordVec
19996,17 ketosteroids urine,36312355,37393636,Urine 17 ketogenic steroid level,0.891024,BioWordVec
19997,17 ketosteroids urine,36312355,4246849,Urine 18-hydroxycortisol level (procedure),0.884305,BioWordVec


In [29]:
# Объединить таблицы, устранить дублирующие строки с одинаковыми парами сорс-код и potential_concept_id
table_name = 'combined_table_'+str(current_date)+'.csv'
df_exp = pd.concat([df_tfidf, df_biowordvec, df_fuzz_ratio])
df_exp = df_exp.sort_values(by=['voc1_name']) 
df_exp

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,similarity,origin,min_distance
4,17 ketosteroids urine,36312355,37393634,Urine ketone test,NaN,Levenshtein_distance,8.0
8,17 ketosteroids urine,36312355,4018340,17-Ketosteroids measurement,NaN,Levenshtein_distance,8.0
7,17 ketosteroids urine,36312355,4042261,Urine 17 ketogenic steroid lev,NaN,Levenshtein_distance,8.0
5,17 ketosteroids urine,36312355,4055829,Urine ketone test = +,NaN,Levenshtein_distance,8.0
3,17 ketosteroids urine,36312355,4150637,Urine ketone test,NaN,Levenshtein_distance,8.0
...,...,...,...,...,...,...,...
4,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer,0.881319,BioWordVec,NaN
6,Vulval cancer recurrent,36617671,4312395,"Cancer of vulva, disseminated",0.860730,BioWordVec,NaN
7,Vulval cancer recurrent,36617671,45769825,Recurrent primary malignant neoplasm of vulva ...,0.859373,BioWordVec,NaN
0,Vulval cancer recurrent,36617671,195197,Primary vulval cancer,0.918315,BioWordVec,NaN


In [30]:
# Группировка и создание поля flag
grouped = df_exp.groupby(['voc1_name', 'voc2_concept_id'])

def combine_origin(group):
    if len(group) > 1:
        group['flag'] = ' / '.join(sorted(group['origin']))
        return group.iloc[:1]  # оставляем только одну строку для объединенных
    else:
        group['flag'] = group['origin'].iloc[0]
        return group

df_exp = grouped.apply(combine_origin).reset_index(drop=True)
df_exp.drop(['origin'], axis=1, inplace=True)

# Вывод результата
df_exp

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,similarity,min_distance,flag
0,17 ketosteroids urine,36312355,4017199,"17-Ketosteroids measurement, total (procedure)",0.809079,NaN,TfIDF
1,17 ketosteroids urine,36312355,4018340,17-Ketosteroids measurement,NaN,8.0,Levenshtein_distance / TfIDF
2,17 ketosteroids urine,36312355,4035824,Disorder of 17-ketosteroids,0.858936,NaN,TfIDF
3,17 ketosteroids urine,36312355,4036649,Increased 17-ketosteroids level,0.791854,NaN,TfIDF
4,17 ketosteroids urine,36312355,4042261,Urine 17 ketogenic steroid lev,NaN,8.0,BioWordVec / Levenshtein_distance
...,...,...,...,...,...,...,...
28537,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer (disorder),0.579575,NaN,BioWordVec / TfIDF
28538,Vulval cancer recurrent,36617671,37208260,Recurrent renal calculus,NaN,9.0,Levenshtein_distance
28539,Vulval cancer recurrent,36617671,40487046,Carcinoma of vulva,0.855377,NaN,BioWordVec
28540,Vulval cancer recurrent,36617671,44808049,Recurrent leg ulcer,NaN,9.0,Levenshtein_distance


In [31]:
result = df_exp.groupby(['flag']).size().sort_values(ascending=False).reset_index(name='count')
result

,flag,count
0,Levenshtein_distance,9294
1,TfIDF,6723
2,BioWordVec,6383
3,BioWordVec / TfIDF,2253
4,BioWordVec / Levenshtein_distance / TfIDF,1631
5,Levenshtein_distance / TfIDF,1375
6,BioWordVec / Levenshtein_distance,883


In [32]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: tensorflow-examples 37df0b9a2f7240831642cea4380e4f7ec33bee83- has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorflow-examples or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [33]:
df_exp = df_exp.drop_duplicates(subset=['voc1_name', 'voc2_concept_id'])
df_exp.to_excel('my_table_30112023.xlsx', index=False)

In [8]:
df_exp = pd.read_excel('my_table_30112023.xlsx')
df_exp

,voc1_name,voc1_concept_id,voc2_concept_id,voc2_name,similarity,min_distance,flag
0,17 ketosteroids urine,36312355,4017199,"17-Ketosteroids measurement, total (procedure)",0.809079,NaN,TfIDF
1,17 ketosteroids urine,36312355,4018340,17-Ketosteroids measurement,NaN,8.0,Levenshtein_distance / TfIDF
2,17 ketosteroids urine,36312355,4035824,Disorder of 17-ketosteroids,0.858936,NaN,TfIDF
3,17 ketosteroids urine,36312355,4036649,Increased 17-ketosteroids level,0.791854,NaN,TfIDF
4,17 ketosteroids urine,36312355,4042261,Urine 17 ketogenic steroid lev,NaN,8.0,BioWordVec / Levenshtein_distance
...,...,...,...,...,...,...,...
28537,Vulval cancer recurrent,36617671,37208178,Recurrent ovarian cancer (disorder),0.579575,NaN,BioWordVec / TfIDF
28538,Vulval cancer recurrent,36617671,37208260,Recurrent renal calculus,NaN,9.0,Levenshtein_distance
28539,Vulval cancer recurrent,36617671,40487046,Carcinoma of vulva,0.855377,NaN,BioWordVec
28540,Vulval cancer recurrent,36617671,44808049,Recurrent leg ulcer,NaN,9.0,Levenshtein_distance


In [10]:
#To download table into database scheme 
login="dev_test5"
passw = "7hGg365$%fhhTfr$dJ"
engine = create_engine('postgresql://{}:{}@ovh07.odysseusinc.com:5555/postgres'.format(login, passw))
table_name = 'am_gpt_meddra_pt_first_2000_051223_out_75_full_variant'
df_exp.to_sql(table_name, engine, if_exists='replace', index=False)
print(f'Import {table_name} was sucessful!')

Import am_gpt_meddra_pt_first_2000_051223_out_75_full_variant was sucessful!


In [15]:
combined_df = df_exp.copy()
combined_df['question'] = 'Pick the best semantic match for '
combined_df = combined_df[['question'] + [col for col in combined_df.columns if col != 'question']]
combined_df.rename(columns={'voc1_name': 'source_code_description', 'voc2_concept_id': 'potential_target_concept_id'}, inplace=True)
combined_df = combined_df.sort_values(by=['source_code_description', 'potential_target_concept_id'])
combined_df = combined_df[['question', 'source_code_description', 'potential_target_concept_id']]
combined_df['chatgptreply']=''
combined_df['target_concept_id'] = None
combined_df['log_id']=None
combined_df['target_concept_id'] = combined_df['target_concept_id'].astype('Int64')
combined_df['log_id'] = combined_df['log_id'].astype('Int64')
combined_df

,question,source_code_description,potential_target_concept_id,chatgptreply,target_concept_id,log_id
0,Pick the best semantic match for,17 ketosteroids urine,4017199,,<NA>,<NA>
1,Pick the best semantic match for,17 ketosteroids urine,4018340,,<NA>,<NA>
2,Pick the best semantic match for,17 ketosteroids urine,4035824,,<NA>,<NA>
3,Pick the best semantic match for,17 ketosteroids urine,4036649,,<NA>,<NA>
4,Pick the best semantic match for,17 ketosteroids urine,4042261,,<NA>,<NA>
...,...,...,...,...,...,...
28537,Pick the best semantic match for,Vulval cancer recurrent,37208178,,<NA>,<NA>
28538,Pick the best semantic match for,Vulval cancer recurrent,37208260,,<NA>,<NA>
28539,Pick the best semantic match for,Vulval cancer recurrent,40487046,,<NA>,<NA>
28540,Pick the best semantic match for,Vulval cancer recurrent,44808049,,<NA>,<NA>


In [16]:
#To download table into database scheme 
login="dev_test5"
passw = "7hGg365$%fhhTfr$dJ"
engine = create_engine('postgresql://{}:{}@ovh07.odysseusinc.com:5555/postgres'.format(login, passw))
table_name = 'am_gpt_meddra_pt_first_2000_051223_out_75'
combined_df.to_sql(table_name, engine, if_exists='replace', index=False)
print(f'Import {table_name} was sucessful!')

Import am_gpt_meddra_pt_first_2000_051223_out_75 was sucessful!
